This week we will be learning about aggregations and using them to do a deeper investigation into the effects of different types of joins. 

We will also do a few problems exploring matrix inverses. 

### Tutorials
- https://www.datacamp.com/community/tutorials/pandas-multi-index
- https://www.datacamp.com/community/tutorials/pandas-split-apply-combine-groupby

## Grouping Data

The expressiveness of Python and pandas allows complex group operations using any function that accepts a pandas object or NumPy array. This can include:

- Splitting a pandas object into pieces using one or more keys
- Calculating group summary statistics
- Applying within-group transformations or other manipulations
- Computing pivot tables and cross-tabulations
- Performing quantile analysis and other statistical group analyses

### GroupBy Operations

Group operations involve the `split-apply-combine` mechanism.

1. Data are split into groups based on one or more keys
2. A function is applied to each group
3. Results of the function application are combined into a new object

Grouping keys can take many forms, and the keys do not have to be all of the same type.

pandas `groupby` method returns a GroupBy object that can be re-used.

DataFrame columns can be used as the group keys.

Numeric aggregations will exclude `nuisance` (non-numeric) columns from the result

By default `groupby` groups on axis=0, but can group on any of the other axes.

### Iterating over groups

The GroupBy object supports iteration, generating a sequence of 2-tuples containing the group name along with the chunk of data.

Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect of column subsetting for aggregation. This means that:
```
df.groupby('key1')['data1']
```
is essentially the same as:
```
df['data1'].groupby(df['key1'])
```

### Reindexing

The output of GroupBys are often multi-indexed. This is typically not the form you want for your analysis. This means that you will want to reset your index or remove multi indexes on your columns. See below for examples of that. 

In [1]:
import pandas as pd
import numpy as np

speeds = pd.DataFrame(
    [
        ("bird", "Falconiformes", 389.0, 0),
        ("bird", "Psittaciformes", 24.0, 0),
        ("mammal", "Carnivora", 80.2, 0),
        ("mammal", "Primates", np.nan, 0),
        ("mammal", "Carnivora", 58, 0),
    ],
    index=["falcon", "parrot", "lion", "monkey", "leopard"],
    columns=("class", "order", "max_speed", 'min_speed'),
)

not_reindexed = speeds.groupby(["class", "order"]).agg(['sum', 'mean'])
not_reindexed

max_speed        min_speed     
                            sum   mean       sum mean
class  order                                         
bird   Falconiformes      389.0  389.0         0  0.0
       Psittaciformes      24.0   24.0         0  0.0
mammal Carnivora          138.2   69.1         0  0.0
       Primates             0.0    NaN         0  0.0

In [2]:
not_reindexed.columns = ["_".join(i) for i in not_reindexed.columns]
not_reindexed

max_speed_sum  max_speed_mean  min_speed_sum  \
class  order                                                          
bird   Falconiformes           389.0           389.0              0   
       Psittaciformes           24.0            24.0              0   
mammal Carnivora               138.2            69.1              0   
       Primates                  0.0             NaN              0   

                       min_speed_mean  
class  order                           
bird   Falconiformes              0.0  
       Psittaciformes             0.0  
mammal Carnivora                  0.0  
       Primates                   0.0

In [3]:
not_reindexed.reset_index()

,class,order,max_speed_sum,max_speed_mean,min_speed_sum,min_speed_mean
0,bird,Falconiformes,389.0,389.0,0,0.0
1,bird,Psittaciformes,24.0,24.0,0,0.0
2,mammal,Carnivora,138.2,69.1,0,0.0
3,mammal,Primates,0.0,NaN,0,0.0


### All Data Descriptions


* `customers` - A table of customers. The unique id for the customer is `customer_id` meaning only one id per customer. 
* `orders` - A table of orders. The unique id for the order is `order_id` there is a foreign key (`customer_id`) that tells you which customer placed the order.
* `web_visits`: Describes visitors to the website, with `visitor_id` be a unique id per visitor to the website. If the visitor is a logged in customer then the `customer_id` has a value otherwise it is null. 
* `ad_clicks`: Describes clicks on ad on an external website, the `external_user_id` is an unique id per user from the external website. If the user is a known in customer then the `customer_id` has a value otherwise it is null. 

In [4]:
import generate_week_10_data

customers, orders, web_visits, ad_clicks = generate_week_10_data.create_data()

### Problem 1 (10 pts)

1. What is the order amount per `customer_name`, only include customers who have orders
2. What is the order amount per `customer_name`, include customers even if they have no orders

#### Grading: 
5 points per question. 2 points for the correct join and 3 point for the aggragations. 

### Problem 2 (5 pts)
If you look at the raw results of the join you can visually see that there are duplicates for the columns that started in the `customer` table. This wasn't a concern above because we we doing an aggregation on a column in the `orders` table. Let's think through problems that require a join but are looking to do an aggregation with a column from the `customers` table. 

1. What is the average `customer_age` for customers who have placed orders. For this problem print the accurate average `customer_age` with duplicates remove and also print the inaccure average `customer_age` without duplicates removed. 
2. What is the average `customer_age` for customers who have placed orders versus those who haven't. 

#### Grading: 
5 points per question. 2 points for the correct join and 3 point for the aggragations. 

### Problem 3 

Per customer what is the total number of web visits and ads clicked on? 

For this problem I want you to start with the `customers_visits_clicks_raw` table below and again do an accurate count that only counts a web visit once and an inaccurate count that counts duplications. Here is what the results should look like: 

<img src="images/number_visits_clicks.png" width="800"/>


#### Grading: 
5 points for joining correctly. 5 points for the correct aggragtions.  


In [5]:
# Start with this dataset 
customers_visits_clicks_raw = customers.merge(web_visits, on="customer_id", how="left").merge(ad_clicks, on="customer_id", how="left")

### Problem 4 (5 pts)

We will now work through a problem with outer joins.

Create a dataframe with the daily number of: 
* web visits
* unique web visitors
* unique customers visiting website
* ads clicks
* unique users clicking
* unique customers clicking

You should include the full date range 2024-05-01 to 2024-05-09 even though not all dates are included in either dataframe. 

You will create this data frame two ways:
1. Join `web_visits` to `ad_clicks` first and then perform the calculations
2. Create aggregate dataframes for `web_visits` to `ad_clicks` first and then join those aggregate dataframes together

#### Grading: 
5 points per method. 2 points for the correct join and 3 point for the aggragations. 

### Problem 5 (15 pts)

Using the same two methods as problem 4 attempt to include:
* mean time on page per date 
* total time on page per date

One method will lead to some incorrect values unless you do a bit more work to correct the values. 

Write out a which method leads to correct values, which leads to incorrect values and why. 

#### Grading: 
* 5 points per method. 2 points for the correct join and 3 point for the aggragations. 
* 5 points for the written explaination. 

### Problem 6 (15 pts)

Implement the full algorithm described in [Inverting Any Square Full-Rank Matrix](https://learning.oreilly.com/library/view/practical-linear-algebra/9781098120603/ch08.html#inverting-any-square) and reproduce Figure 8-3. Of course, your matrices will look different from Figure 8-3 because of random numbers, although the grid and identity matrices will be the same.

#### Grading: 
* 10 points for implimenting the algorithm. 
* 5 points for reproducing the figure. 

### Problem 7 (15 pts)

The LIVE EVIL rule applies to the inverse of multiplied matrices. Test this in code by creating two square full-rank matrices $A$ and $B$, then use Euclidean distance to compare:
1. $(AB)^{-1}$ 
2. $A^{-1}B^{-1}$
3. $B^{-1}A^{-1}$

Before starting to code, make a prediction about which results will be equal. Print out your results using formatting like the following:

```
Distance between (AB)^-1 and (A^-1)(B^-1) is ___
Distance between (AB)^-1 and (B^-1)(A^-1) is ___
Distance between (A^-1)(B^-1) and (B^-1)(A^-1) is ___
```

#### Grading: 
5 points per comparision. 